In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -y psi4 python=3.7 -c psi4

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

--2022-04-19 00:38:03--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-19 00:38:03--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M  67.3MB/s    in 1.1s    

2022-04-19 00:38:04 (67.3 MB/s) - ‘Miniconda3-latest-Linu

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [3]:
psi4.set_output_file('log.txt')

mol = psi4.geometry("""
symmetry c1
0 1
O       -1.0239104333      1.3152089264     -0.0149071027                 
H       -0.0345840622      1.2807129464      0.0105245830                 
H       -1.3132760473      0.5529382075      0.5470664770                 
""")

# input (PCM Solver)
pcm = '''
units = angstrom
medium
{
    solvertype = iefpcm
    solvent = Water
}
cavity
{
    type = gepol
    scaling = True
    radiiset = uff
    mode = implicit
}
'''
psi4.set_options({'pcm': True})
psi4.pcm_helper(pcm)

scf_e, wfn = psi4.energy("B3LYP/6-31G", return_wfn = True)
print('energy at the {} level of theory:\t{:.4f}'.format("B3LYP/6-31G", scf_e))

energy at the B3LYP/6-31G level of theory:	-76.3928


In [4]:
import numpy as np
import pandas as pd

df = pd.DataFrame()
psi4.oeprop(wfn, 'MULLIKEN_CHARGES')
df['B3LYP/6-31G'] = np.array(wfn.atomic_point_charges())
print (df)

   B3LYP/6-31G
0    -0.753415
1     0.376708
2     0.376708


In [5]:
psi4.oeprop(wfn,'DIPOLE')
dipole_vec = psi4.variable('scf dipole')
dipole = np.sqrt(np.sum(dipole_vec ** 2))
print(f'{dipole:.3f} [D]')

1.033 [D]


In [6]:
psi4.oeprop(wfn, 'WIBERG_LOWDIN_INDICES')
!cat log.txt


Scratch directory: /tmp/

Scratch directory: /tmp/
  PCM does not make use of molecular symmetry: further calculations in C1 point group.

*** tstart() called on e7e321291ab6
*** at Tue Apr 19 00:41:37 2022

   => Loading Basis Set <=

    Name: 6-31G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   117 file /usr/local/share/psi4/basis/6-31g.gbs 
    atoms 2-3 entry H          line    26 file /usr/local/share/psi4/basis/6-31g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RKS Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center        